In [1]:
import random
import hashlib

In [3]:
def gcd(a,b): 
    if b==0: 
        return a 
    else: 
        return gcd(b,a%b)

In [5]:
 def isPrime(n) : 
    # Corner cases 
    if (n <= 1) : 
        return False
    if (n <= 3) : 
        return True
   
    # This is checked so that we can skip  
    # middle five numbers in below loop 
    if (n % 2 == 0 or n % 3 == 0) : 
        return False
   
    i = 5
    while(i * i <= n) : 
        if (n % i == 0 or n % (i + 2) == 0) : 
            return False
        i = i + 6
   
    return True

In [12]:
 # Get a prime number
 def generatePrime(num = 100):
    L1 = []
    for i in range(60, num + 1): 
        if isPrime(i):
            L1.append(i)
    
    p = random.choice(L1)
    L1.pop(L1.index(p))
    q = random.choice(L1)
    
    t = (p-1)*(q-1)
    n = p*q
    
    for e in range(2,t): 
        if gcd(e,t)== 1: 
            break
    for i in range(1,10): 
        x = 1 + i*t 
        if x % e == 0: 
             d = int(x/e) 
             break
    
    return e,d,n


In [20]:
Alphabet_List = {'A': '01', 'B': '02', 'C': '03', 'D': '04', 'E': '05', 'F': '06', 
                 'K': '11', 'L': '12', 'M': '13', 'N': '14', 'O': '15', 'P': '16', 
                 'U': '21', 'V': '22', 'W': '23', 'X': '24', 'Y': '25', 'Z': '26', 
                 '4': '31', '5': '32', '6': '33', '7': '34', '8': '35', '9': '36', '0': '37'}

key_list = list(Alphabet_List.keys()) 
val_list = list(Alphabet_List.values()) 

def convertText(msg, Ekey, N):
    li = list(msg)
    lii = [Alphabet_List[i] for i in li if i in Alphabet_List]
    
    if len(lii) % 2 != 0:
        lii.append('27')  # Padding with '27' if odd length

    l1 = [int(lii[i] + lii[i+1]) for i in range(0, len(lii), 2)]
    
    ctt = [str(pow(no, Ekey, N)).zfill(4) for no in l1]  
    ct = ''.join(ctt)
    return ct

def decrypt(cipherText, Dkey, N):
    text = [str(pow(int(cipherText[i:i+4]), Dkey, N)).zfill(4) for i in range(0, len(cipherText), 4)]
    
    L1 = []
    for i in text:
        L1.append(i[0:2])
        L1.append(i[2:4])

    L2 = []
    for i in L1:
        if i in val_list:
            L2.append(key_list[val_list.index(i)])
        else:
            L2.append('?')  # Placeholder for missing mappings

    msg = ''.join(L2).replace('?', '')  # Removing unknown characters
    return msg


In [25]:
import socket
import threading

# Function to generate prime numbers (assuming it's defined elsewhere)
def generatePrime():
    return 17, 2753, 3233  # Example RSA key values (e, d, n)

e, d, n = generatePrime()

localIP = "127.0.0.2"
localPort = 2004
bufferSize = 1024

msgFromServer = "Hello UDP Client " + str(e) + " " + str(n)
flag = 1
bytesToSend = str.encode(msgFromServer)

# Create a UDP socket
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# Bind to address and port
UDPServerSocket.bind((localIP, localPort))
print("UDP server up and listening")

def recv():
    global flag
    while True:
        receive = UDPServerSocket.recvfrom(bufferSize)
        msg = receive[0].decode('utf-8')
        plainText = decrypt(msg, d, n)

        if msg.lower() == 'bye':
            flag = 0
            break

        print(plainText)

def Send(client_address):
    global flag
    while True:
        if flag == 0:
            print("Connection closed")
            break

        message = input("Enter your reply: ").upper()
        parts = message.split()

        if len(parts) < 3:
            print("Invalid input format. Use: MESSAGE EKEY N")
            continue

        cipherText = convertText(parts[0], int(parts[1]), int(parts[2]))
        UDPServerSocket.sendto(cipherText.encode('utf-8'), client_address)

# Listen for incoming messages
while True:
    flag = 1
    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
    message = bytesAddressPair[0].decode('utf-8')
    address = bytesAddressPair[1]

    print(f"Message from Client: {message}")
    print(f"Client IP Address: {address}")

    UDPServerSocket.sendto(bytesToSend, address)

    threading.Thread(target=recv).start()
    threading.Thread(target=Send, args=(address,)).start()


UDP server up and listening
Message from Client: Hello UDP Server!. My public Key is: 17 3233
Client IP Address: ('127.0.0.1', 61856)


Enter your reply:  Hello 5 5293


Exception in thread Thread-5 (recv):
Traceback (most recent call last):
  File "C:\Users\yadne\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\yadne\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\yadne\AppData\Local\Temp\ipykernel_16484\1689126408.py", line 28, in recv
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


Enter your reply:  Hello 5 5293


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host